<a href="https://colab.research.google.com/github/faisalalh1122/BootCampT5/blob/main/Copy_of_RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 7.46MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [3]:
!pip install langchain
!pip install transformers torch chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) 

In [4]:
!pip install langchain_community
!pip install sentence_transformers
!pip install OpenAI
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [5]:
import pandas as pd
df=pd.read_csv('Dataset.csv')


In [6]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [8]:
df.shape

(104, 2)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [10]:
df.isnull().sum()

,0
المخالفة,0
الغرامة,0


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [28]:
df

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


In [56]:

max_length = df['المخالفة'].str.len().max()
print(max_length)


141


In [163]:
df.sample()

,المخالفة,الغرامة
39,عدم إعطاء الأفضلية للمركبات التي على الطريق ال...,الغرامة المالية 500 - 900 ريال


## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [136]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader

splitter = RecursiveCharacterTextSplitter(chunk_size=50,chunk_overlap=10)
loader = DataFrameLoader(df, page_content_column="المخالفة")
documents = loader.load()
chunks = splitter.split_documents(documents)

In [137]:
chunks[15]

Document(metadata={'الغرامة': 'الغرامة المالية 150 - 300 ريال'}, page_content='أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.')

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [138]:
!pip install chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [139]:
embeddings = HuggingFaceEmbeddings(model_name="asafaya/bert-base-arabic")
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [178]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = """
اجب على السؤال المدخل باللغه العربيه وفقا للقانون السعودي والالتزام بالنص المذكور
{السؤال}

وفقا للاتي :
{النص}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

print(prompt)

input_variables=['السؤال', 'النص'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['السؤال', 'النص'], input_types={}, partial_variables={}, template='\nاجب على السؤال المدخل باللغه العربيه وفقا للقانون السعودي والالتزام بالنص المذكور \n{السؤال}\n\nوفقا للاتي :\n{النص}\n'), additional_kwargs={})]


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [141]:
pip install langchain-groq

In [179]:
from langchain_groq import ChatGroq

In [180]:
GROQ_API_KEY="*****"

In [181]:
# chat = ChatGroq(temperature=0.1, model_name="mixtral-8x7b-32768",api_key=GROQ_API_KEY )

# bad results!

In [182]:

chat = ChatGroq(temperature=0.5,  model_name="llama3-8b-8192",api_key=GROQ_API_KEY )

# As well this , is it because overlap and size or because the model ??

In [123]:
# chat = ChatGroq(temperature=0.5,  model_name="asafaya/bert-base-arabic",api_key=GROQ_API_KEY )

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [183]:
df.sample(1)

,المخالفة,الغرامة
68,عدم ارتداء الخوذة أثناء قيادة الدراجة الآلية.,الغرامة المالية 1000 - 2000 ريال


In [184]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="وضع حواجز داخل المركبة أو خارجها تعوق رؤية السائق"),
    HumanMessage(content="ماهي الغرامة ؟"),
]

chat.invoke(messages)

AIMessage(content='الغرامة في هذا الحالة هي:\n\n"تخلف عن رؤية السائق" أو "تخلف عن الرؤية" أو "تخلف عن الرؤية الخارجية"، وهي عقوبة تُقدم على السائقين الذين يضعون حواجز داخل المركبة أو خارجها، مما يتعارض مع رؤية السائق أو يحد من رؤية الطريق أمامهم.\n\nهذه العقوبة تُقدم لمن يخالف القانون رقم 51 لسنة 1995 المتعلق بالمرور، المادة 49 منه، التي تنص على أن "المركبة يجب أن تملك رؤية خارجية كافية لمراقبة الطريق والمرور على الطريق، ولا يجوز وضع أي حواجز أو منشآت داخل أو خارج المركبة تتعارض مع رؤية السائق أو تحد من رؤية الطريق".', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 201, 'prompt_tokens': 41, 'total_tokens': 242, 'completion_time': 0.1675, 'prompt_time': 0.00494581, 'queue_time': 0.009528588, 'total_time': 0.17244581}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-0665b4ed-e590-46a4-a8ec-aaa0914ca4d2-0', usage_metadata={'input_tokens': 41, 'output_tokens': 201, 'total_tokens': 242}

In [185]:
from langchain.chains import LLMChain

In [186]:
model = LLMChain(llm=chat, prompt=prompt)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [118]:
#def query_rag(query):
 #   docs = vectorstore.similarity_search(query)
  #  context = "\n".join([doc.page_content for doc in docs])

In [187]:
def query_rag(query):
  query_result = vectorstore.as_retriever()
  response = model.invoke({"النص": query_result, "السؤال": query})
  return response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [191]:
question = "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟"
result=query_rag(question)
result

{'النص': VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x793146345c60>, search_kwargs={}),
 'السؤال': 'ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟',
 'text': 'وفقاً для القانون السعودي، فإن عدم الوقوف وقوفاً تاماً عند إشارة هو من الأعمال المخالفة التي تحددها المادة (38) من القانون السعودي للطريق السريع، والتيstates أن:\n\n"من يخالف إشارة المرور يلزمه غرامة مالية تتراوح بين مائة ريال وخمسة آلاف ريال، ويمكن أن يلزمه أيضاً سجلاً في سجل المخالفين."\n\nلذلك، عقوبة عدم الوقوف وقوفاً تاماً عند إشارة حسب القانون السعودي هي غرامة مالية تتراوح بين مائة ريال وخمسة آلاف ريال، بالإضافة إلى سجلاً في سجل المخالفين.'}

In [189]:
df.sample(1)

,المخالفة,الغرامة
84,تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر.,الغرامة المالية 3000 - 6000 ريال


In [190]:
question = " ماهي غرامة تجاوز المرور الضوئيه اثناء الضوء الاحمر ؟"
result=query_rag(question)
result

{'النص': VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x793146345c60>, search_kwargs={}),
 'السؤال': ' ماهي غرامة تجاوز المرور الضوئيه اثناء الضوء الاحمر ؟',
 'text': 'وفقاً للقانون السعودي، فإن الغرامة ل تجاوز المرور الضوئي أثناء الضوء الأحمر هي 500 ريال سعودي، وذلك طبقاً لما جاء في المادة 39 من القانون رقم 28 لسنة 1990 بشأن المرور.\n\nكما أن المادة 40 من القانون نفسه جاءت بذلك: "يحرم من المرور لمدة شهرين في الحالات التالية:\n\n1. تجاوز المرور الضوئي أثناء الضوء الأحمر.\n2. تجاوز المرور الضوئي أثناء الضوء الأخضر.\n3. تجاوز المرور الضوئي أثناء وجود الضوء الأصفر.\n\nويمكن أن يلزم المحكوم بالغرامة لمدة تصل إلى 1000 ريال سعودي في الحالات التي يصدر فيها قرار بالغرامة."'}